In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as si

def chebD(n):
    """
    Computes the (n+1)x(n+1) spectral differentiation matrix\
    using Chebyshev roots according to Ch 6 of Trefethen's\
    "Spectral methods in MATLAB". Returns D and nodes on [-1, 1].
    Note that they are ordered backwards, i.e. 1 to -1!
    """
    if n == 0:
        x = 1; D = 0; w = 0
    else:
        a = np.linspace(0.0, np.pi, n+1)
        x = np.cos(a)
        b = np.ones_like(x)
        b[0] = 2; b[-1] = 2
        d = np.ones_like(b)
        d[1::2] = -1
        c = b*d
        X = np.outer(x, np.ones(n+1))
        dX = X - X.T
        D = np.outer(c, 1/c) / (dX + np.identity(n+1))
        D = D - np.diag(D.sum(axis=1))
    return D, x

# Spectral solver

om = 100 # frequency parameter

def solve(n):
    """
    Encodes the ODE and initial conditions with (n+1)-node collocation, and solves the\
    rectangular system with numpy's least squares method.
    """
    D, x = chebD(n)
    lhs = np.zeros((n+2, n+1))
    lhs[:-1, :] = D @ D + om**2*(np.identity(n+1))
    lhs[-2, :] = 0; lhs[-2, -1] = 1
    lhs[-1, :] = D[-1]
    rhs = np.zeros(n+2)
    rhs[-2] = 1
    rhs[-1] = 0
    u, res, rank, sing = np.linalg.lstsq(lhs, rhs)
    return u

def solve_spectral():
    """
    Repeatedly calls the spectral solver with double the nodes, comparing the answer at the end of\
    the solution interval until the tolerance is reached, or a certain n is exceeded.
    """
    tol = 1e-12
    n_ini = 16
    n = n_ini
    u = solve(n)
    err = 1
    while err > tol and n <= 256: # The second condition here is just to guard against an infinite loop
        u_new = solve(2*n)
        err = np.abs(u[0] - u_new[0])
        u = u_new; n *= 2;
    return u[0]
    
%time solve_spectral()

# Scipy's default Runge-Kutta solver

def f_ivp(t, u):
    """ The right-hand-side in u' = f(t, u). """
    du = np.zeros(2)
    du[0] = u[1]
    du[1] = - om**2*u[0]
    return du

u0 = np.array([1, 0])
tspan = (-1, 1)

%time solution = si.solve_ivp(f_ivp, tspan, u0, rtol = 1e-12, atol = 1e-12)

solution.t[-1], solution.y[0, -1]